<a href="https://colab.research.google.com/github/samipn/unsloth.ai_demo/blob/main/colab5_continued_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 5 — Continued Pretraining (teach a new language) + resume/export
Show CPT on raw text to extend vocabulary/knowledge, then resume from a LoRA/adapter checkpoint and export to Ollama.

In [1]:
#@title Install Unsloth + deps (Colab-safe)
%pip -q install --upgrade pip
%pip -q install unsloth datasets trl transformers accelerate bitsandbytes peft --no-cache-dir
import torch, platform
print("PyTorch:", torch.__version__, "CUDA:", torch.version.cuda, "Python:", platform.python_version())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
PyTorch: 2.8.0+cu126 CUDA: 12.6 Python: 3.12.12


In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, textwrap

MODEL_NAME = "HuggingFaceTB/SmolLM2-135M"  # tiny model good for CPT demo
max_seq_length = 2048
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME, max_seq_length=max_seq_length, dtype=dtype, load_in_4bit=False
)

# Suppose we want to teach a tiny constructed language ("MiniLang")
raw_corpus = """
<doc>
mi wile moku. sina pona!
tomo li suli. jan li moku e kili.
tenpo ni la mi tawa tomo. sina lon seme?
</doc>
<doc>
moku pona li lon kulupu ni. toki sina li pona mute.
</doc>
"""

# Extend tokenizer if many new tokens
added = tokenizer.add_tokens(["mi","sina","pona","moku","tomo","jan","kili","tenpo","ni","la","tawa","lon","seme","kulupu","toki"], special_tokens=False)
model.resize_token_embeddings(len(tokenizer))
print("Added tokens:", added)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM2-135M does not have a padding token! Will use pad_token = <|endoftext|>.


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Added tokens: 15


In [3]:
# Build raw text dataset for CPT
from datasets import Dataset

lines = [l.strip() for l in raw_corpus.splitlines() if l.strip()]
dataset = Dataset.from_dict({"text": lines})
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_raw, eval_raw = dataset["train"], dataset["test"]

# Simple chunking function
def pack(examples):
    return examples

train_raw = train_raw.map(pack, batched=True)


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [4]:
# Train with TRL SFTTrainer in plain text mode (text completion)
from trl import SFTTrainer, SFTConfig

cfg = SFTConfig(
    output_dir="outputs_cpt_smollm2",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    max_seq_length=512,
    logging_steps=10,
    save_steps=200,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_raw,
    eval_dataset=eval_raw,
    dataset_text_field="text",
    args=cfg,
)
trainer.train()
trainer.save_model("smollm2_cpt")
tokenizer.save_pretrained("smollm2_cpt")


num_proc must be <= 7. Reducing num_proc to 7 for dataset of size 7.


Unsloth: Tokenizing ["text"] (num_proc=7):   0%|          | 0/7 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/1 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7 | Num Epochs = 3 | Total steps = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 106,203,456 of 134,520,768 (78.95% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samipn (samipn-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss


('smollm2_cpt/tokenizer_config.json',
 'smollm2_cpt/special_tokens_map.json',
 'smollm2_cpt/vocab.json',
 'smollm2_cpt/merges.txt',
 'smollm2_cpt/added_tokens.json',
 'smollm2_cpt/tokenizer.json')

In [5]:
# Resume from a LoRA adapter (continued finetuning)
# Demonstrate loading a prior adapter and resuming (if available).
from pathlib import Path
adapter_dir = Path("smollm2_lora_adapter")
if adapter_dir.exists():
    from peft import PeftModel
    model = PeftModel.from_pretrained(model, adapter_dir.as_posix())
    print("Loaded previous LoRA adapter; you can trainer.train() again to continue.")
else:
    print("No prior adapter found in this session; skip resume demo.")


No prior adapter found in this session; skip resume demo.


In [6]:
# Export to Ollama (auto Modelfile) after merge to 16-bit
from unsloth import FastLanguageModel as FLM
try:
    merged = FLM.merge_and_unload(model)
    merged.save_pretrained("smollm2_cpt_merged_16bit")
    tokenizer.save_pretrained("smollm2_cpt_merged_16bit")
    print("Merged model ready. To create Modelfile in Python, see Unsloth docs.")
except Exception as e:
    print("Merge skipped:", e)


Merge skipped: type object 'FastLanguageModel' has no attribute 'merge_and_unload'


In [7]:
# Minimal inference in the new language (qualitative)
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)
prompt = "toki: sina pona la mi pona. o toki e ijo."
inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=80)
print(tokenizer.decode(out[0], skip_special_tokens=True))


toki: sina pona la mi pona. o toki e ijo.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o  elegans  elegans.  o 
